In [1]:
import pandas as pd
import numpy as np
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from transformers import RobertaModel, RobertaTokenizer

# Load the model
model_name = "roberta-large"
model = RobertaModel.from_pretrained(model_name)

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_name)
from datasets import load_dataset
dataset = load_dataset("yaful/DeepfakeTextDetect")
import random
seed_value = 42
torch.manual_seed(seed_value)
np.random.seed(seed_value)

random.seed(seed_value)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
len(dataset['train'][0]['src'])

9

In [13]:
unique_sources = []
for i in range(len(dataset['train'])):
    source = dataset['train'][i]['src']
    if source not in unique_sources:
        unique_sources.append(source)
        

In [18]:
for i in range(len(unique_sources)):
    if 'squad' in unique_sources[i]:
        print(unique_sources[i])

squad_human
squad_machine_continuation_opt_1.3b
squad_machine_continuation_opt_iml_30b
squad_machine_continuation_7B
squad_machine_continuation_65B
squad_machine_continuation_t0_3b
squad_machine_continuation_flan_t5_small
squad_machine_continuation_t0_11b
squad_machine_continuation_flan_t5_xxl
squad_machine_continuation_opt_30b
squad_machine_continuation_gpt-3.5-trubo
squad_machine_continuation_text-davinci-002
squad_machine_continuation_flan_t5_large
squad_machine_continuation_gpt_neox
squad_machine_continuation_gpt_j
squad_machine_continuation_flan_t5_base
squad_machine_continuation_opt_125m
squad_machine_continuation_flan_t5_xl
squad_machine_continuation_GLM130B
squad_machine_continuation_bloom_7b
squad_machine_continuation_opt_13b
squad_machine_continuation_30B
squad_machine_continuation_opt_2.7b
squad_machine_continuation_opt_6.7b
squad_machine_continuation_opt_350m
squad_machine_continuation_opt_iml_max_1.3b
squad_machine_continuation_13B
squad_machine_continuation_text-davinci-0

In [19]:
llama_sources = ["squad_machine_continuation_7B",
"squad_machine_continuation_65B","squad_machine_continuation_30B","squad_machine_continuation_13B"]

In [21]:
human_train = [data for data in dataset['train'] if data['src'] == 'squad_human']
human_test = [data for data in dataset['test'] if data['src'] == 'squad_human']
human_val = [data for data in dataset['validation'] if data['src'] == 'squad_human']

llm_train = [data for data in dataset['train'] if any(l in data['src'] and 'squad' in data['src'] for l in llama_sources)]
llm_test = [data for data in dataset['test'] if any(l in data['src'] and 'squad' in data['src'] for l in llama_sources)]
llm_val = [data for data in dataset['validation'] if any(l in data['src'] and 'squad' in data['src'] for l in llama_sources)]

import random

min_train_length = min(len(human_train), len(llm_train))
min_test_length = min(len(human_test), len(llm_test))
min_val_length = min(len(human_val), len(llm_val))

squad_human_train = random.sample(human_train, min_train_length)
squad_human_test = random.sample(human_test, min_test_length)
squad_human_val = random.sample(human_val, min_val_length)

squad_gpt_train = random.sample(llm_train, min_train_length)
squad_gpt_test = random.sample(llm_train, min_test_length)
squad_gpt_val = random.sample(llm_val, min_val_length)

print(len(human_train), len(llm_train))
print(len(squad_human_train), len(squad_gpt_train))
print(len(squad_human_test), len(squad_gpt_test))
print(len(squad_human_val), len(squad_gpt_val))

15820 3102
3102 3102
389 389
386 386


In [22]:
import pandas as pd

Human = pd.DataFrame()
squad_human = squad_human_train + squad_human_val + squad_human_test
Human['text'] = [data['text'] for data in squad_human]

AI = pd.DataFrame()
squad_gpt = squad_gpt_train + squad_gpt_val + squad_gpt_test
AI['text'] = [data['text'] for data in squad_gpt]

In [23]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to('cuda')
def generate_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = {name: tensor.to('cuda') for name, tensor in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():
        outputs = model(**inputs)
        #embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling of token embeddings
        last_hidden_states = outputs.last_hidden_state
        embeddings = last_hidden_states[:,0,:].to('cpu')
    return embeddings.numpy()
import numpy as np

AI['embeddings'] = AI['text'].apply(generate_bert_embeddings)
Human['embeddings'] = Human['text'].apply(generate_bert_embeddings)

In [24]:
Human_embeddings = Human['embeddings']
AI_embeddings = AI['embeddings']
reshaped_gpt = []
for i in range(len(AI_embeddings)):
    reshaped = AI_embeddings[i].reshape(1,32,32)
    reshaped_gpt.append(reshaped)
    
reshaped_human = []
for i in range(len(Human_embeddings)):
    reshaped = Human_embeddings[i].reshape(1,32,32)
    reshaped_human.append(reshaped)
    
import torch

reshaped_gpt_tensor = torch.tensor(reshaped_gpt)
reshaped_human_tensor = torch.tensor(reshaped_human)

import torch

# Create tensors with zeros and ones
zeros_tensor = torch.zeros(len(reshaped_gpt))
ones_tensor = torch.ones(len(reshaped_human))

# Concatenate the tensors along the first dimension
labels = torch.cat((zeros_tensor, ones_tensor), dim=0)

# Display the result tensor
print(labels)
embeds = torch.cat((reshaped_gpt_tensor, reshaped_human_tensor), dim=0)

print(len(reshaped_gpt_tensor), len(reshaped_human_tensor), len(embeds), len(labels))

tensor([0., 0., 0.,  ..., 1., 1., 1.])
3877 3877 7754 7754


/tmp/ipykernel_302663/4034461053.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/torch/csrc/utils/tensor_new.cpp:245.)
  reshaped_gpt_tensor = torch.tensor(reshaped_gpt)


In [25]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

test_size = 0.2
dataset = TensorDataset(embeds, labels)

# Split dataset into train and test sets
train_dataset, test_dataset = train_test_split(dataset, test_size=test_size, random_state=42)

# Define batch size
batch_size = 64

# Create DataLoader for training set
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
import torch
import torch.nn as nn
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, 8, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(8, 2, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # self.conv3 = nn.Conv2d(4, 128, kernel_size=3, stride=1, padding=1)
        # self.relu3 = nn.ReLU()
        # self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(128, 64)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(64, 32)
        self.relu5 = nn.ReLU()

        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        # x = self.pool3(self.relu3(self.conv3(x)))
        # print(x.shape)
        x = self.flatten(x)
        # print(x.shape)
        x = self.relu4(self.fc1(x))
        x = self.relu5(self.fc2(x))
        x = self.fc3(x) 

        return x

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN(in_channels=1, num_classes=2).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
max_val_accuracy = 0
# Step 5: Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).long()
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    
    # Validation
    model.eval()
    correct = 0
    total = 0
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device).long()
            outputs = model(inputs)
            predicted = torch.argmax(outputs, 1)
            TP += ((predicted == 1) & (labels == 1)).sum().item()
            FP += ((predicted == 1) & (labels == 0)).sum().item()
            TN += ((predicted == 0) & (labels == 0)).sum().item()
            FN += ((predicted == 0) & (labels == 1)).sum().item()
            total += labels.size(0)
            #print(labels, predicted)
            correct += (predicted == labels).sum().item()
    val_accuracy = correct / total 
    if max_val_accuracy<val_accuracy:
        max_TP = TP
        max_FP = FP
        max_TN = TN
        max_FN = FN
    max_val_accuracy = max(max_val_accuracy, val_accuracy)
    
    
    print(f'Validation Accuracy: {val_accuracy:.4f}')
    print(f'TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}')


Epoch [1/100], Loss: 0.6177
Validation Accuracy: 0.8652
TP: 718, FP: 157, TN: 624, FN: 52
Epoch [2/100], Loss: 0.3451
Validation Accuracy: 0.8794
TP: 743, FP: 160, TN: 621, FN: 27
Epoch [3/100], Loss: 0.3159
Validation Accuracy: 0.8852
TP: 742, FP: 150, TN: 631, FN: 28
Epoch [4/100], Loss: 0.3016
Validation Accuracy: 0.8788
TP: 689, FP: 107, TN: 674, FN: 81
Epoch [5/100], Loss: 0.2846
Validation Accuracy: 0.8981
TP: 738, FP: 126, TN: 655, FN: 32
Epoch [6/100], Loss: 0.2749
Validation Accuracy: 0.9046
TP: 743, FP: 121, TN: 660, FN: 27
Epoch [7/100], Loss: 0.2547
Validation Accuracy: 0.9052
TP: 750, FP: 127, TN: 654, FN: 20
Epoch [8/100], Loss: 0.2417
Validation Accuracy: 0.9168
TP: 741, FP: 100, TN: 681, FN: 29
Epoch [9/100], Loss: 0.2324
Validation Accuracy: 0.9039
TP: 703, FP: 82, TN: 699, FN: 67
Epoch [10/100], Loss: 0.2267
Validation Accuracy: 0.9194
TP: 741, FP: 96, TN: 685, FN: 29
Epoch [11/100], Loss: 0.2178
Validation Accuracy: 0.9155
TP: 698, FP: 59, TN: 722, FN: 72
Epoch [12/1

In [27]:
print("Maximum Validation Accuracy: ", max_val_accuracy)
print("TP: ", max_TP)
print("FP: ", max_FP)
print("TN: ", max_TN)
print("FN: ", max_FN)

import math

MCC = (max_TP * max_TN - max_FP * max_FN) / math.sqrt((max_TP + max_FP) * (max_TP + max_FN) * (max_TN + max_FP) * (max_TN + max_FN))

print(f'Matthews Correlation Coefficient (MCC): {MCC:.4f}')

TPR = max_TP / (max_TP + max_FN)
FPR = max_FP / (max_FP + max_TN)

print(f'True Positive Rate (TPR): {TPR:.4f}')
print(f'False Positive Rate (FPR): {FPR:.4f}')


Maximum Validation Accuracy:  0.9361702127659575
TP:  749
FP:  78
TN:  703
FN:  21
Matthews Correlation Coefficient (MCC): 0.8748
True Positive Rate (TPR): 0.9727
False Positive Rate (FPR): 0.0999


In [28]:
AI_embeddings = AI['embeddings']

with open('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code/Deepfake_dataset/Squad/BERT/llama_embeddings.npy', 'wb') as f:
    #for i in range(len(gpt['embeddings'])):
    np.save(f, AI_embeddings)